## Importing the libraries

In [182]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [183]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\minam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [184]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Data Pre-processing

In [186]:
news_dataset = pd.read_csv('news.csv')

In [187]:
news_dataset.shape

(20800, 5)

In [188]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [189]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [190]:
# replacing null values with empty string
news_dataset = news_dataset.fillna('')

In [191]:
news_dataset['author_title_text'] = news_dataset['author'] + ' ' + news_dataset['title'] + ' ' + news_dataset['text']

In [192]:
print(news_dataset['author_title_text'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799    David Swanson What Keeps the F-35 Alive   Davi...
Name: author_title_text, Length: 20800, dtype: object


In [193]:
X = news_dataset.drop(columns = 'label', axis=1)
Y = news_dataset['label']

In [194]:
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [195]:
print(Y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


## Stemming 

In [197]:
port_stem = PorterStemmer()

In [198]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [199]:
news_dataset['author_title_text'] = news_dataset['author_title_text'].apply(stemming)

In [200]:
print(news_dataset['author_title_text'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2        consortiumnew com truth might get fire truth m...
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799    david swanson keep f aliv david swanson author...
Name: author_title_text, Length: 20800, dtype: object


In [201]:
#separating the data and label
X = news_dataset['author_title_text'].values
Y = news_dataset['label'].values

In [202]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet hous dem aid even see comey letter jason chaffetz tweet darrel lucu octob subscrib jason chaffetz stump american fork utah imag courtesi michael jolley avail creativ common licens apolog keith olbermann doubt worst person world week fbi director jame comey accord hous democrat aid look like also know second worst person well turn comey sent infam letter announc fbi look email may relat hillari clinton email server rank democrat relev committe hear comey found via tweet one republican committe chairmen know comey notifi republican chairmen democrat rank member hous intellig judiciari oversight committe agenc review email recent discov order see contain classifi inform long letter went oversight committe chairman jason chaffetz set polit world ablaz tweet fbi dir inform fbi learn exist email appear pertin investig case reopen jason chaffetz jasoninthehous octob cours know case comey actual say review email light unrel 

In [203]:
print(Y)

[1 0 1 ... 0 1 1]


In [204]:
# converting text data to numerical
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [227]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5144938 stored elements and shape (20800, 111501)>
  Coords	Values
  (0, 325)	0.04351705682996119
  (0, 521)	0.022974569956367194
  (0, 635)	0.038727100898403345
  (0, 873)	0.01573563765182704
  (0, 922)	0.016392735476282342
  (0, 1298)	0.021078125206502117
  (0, 1610)	0.01797988302277085
  (0, 1886)	0.11545306338629945
  (0, 3022)	0.046620910610995464
  (0, 3042)	0.01902260509613337
  (0, 3377)	0.011797233080872101
  (0, 3743)	0.030879171097770226
  (0, 4156)	0.016679444655381123
  (0, 4260)	0.01942127157583213
  (0, 4306)	0.026424556938478543
  (0, 4588)	0.01769571497502257
  (0, 4772)	0.025939050637307268
  (0, 4822)	0.041683522324264644
  (0, 4839)	0.014401489020964033
  (0, 6421)	0.018224972680710137
  (0, 6964)	0.02064947765731587
  (0, 8556)	0.021643407923861314
  (0, 9072)	0.014237130761490214
  (0, 10846)	0.04758422449372864
  (0, 12251)	0.02777858540400552
  :	:
  (20799, 106830)	0.03658229119526847
  (20799, 10696

## splitting the dataset

In [230]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

## Model Training

In [233]:
model = LogisticRegression()

In [235]:
model.fit(X_train, Y_train)

LogisticRegression()

## Evaluation

In [238]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [246]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9785456730769231


In [242]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [248]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9526442307692308


## predict from a single input

In [312]:
i=3
X_new = X_test[i]

prediction = model.predict(X_new)
print('true label : ', Y_test[i])
print('predicted : ', prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

true label :  0
predicted :  [1]
The news is Fake
